<a href="https://colab.research.google.com/github/Ritoke/Python/blob/main/AnnModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving breast-cancer-wisconsin.data to breast-cancer-wisconsin.data


In [ ]:
import time
import random
import numpy as np
#import matplotlib.pyplot as plt
#from tensorflow.keras.datasets import mnist
#from keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

Neural Network Class

In [ ]:

class NueralNetwork():
    def __init__(self, layer_param):
        self.layer_param = layer_param
        #dictionary for weight and biases
        self.networkParameters={}
        self.bestNetParameter={}
        self.bestLoss = float('inf')
        #get number of layers in Neural network
        self.networkLength = len(layer_param)
        #intilialize network parameters for all layers excluding the input layer
        for l in range(1, self.networkLength):
            self.networkParameters['W' + str(l)] = np.random.randn(layer_param[l],layer_param[l-1]) / np.sqrt(layer_param[l-1]) #*0.01
            self.networkParameters['b' + str(l)] = np.zeros((layer_param[l],1))

    def softmax(self,z):
      expZ = np.exp(z)
      return expZ/(np.sum(expZ, 0))
    def sigmoid(self,Z):
        A = 1/(1+np.exp(-Z))
        return A
    def relu(self,Z):
        A = np.maximum(0,Z)
        return A
    def derivative_relu(self,Z):
        return np.array(Z > 0, dtype = 'float')

    def forward_propagation(self,X, activation):
        forward_cache = {}
        L = len(self.networkParameters) // 2

        forward_cache['A0'] = X

        for l in range(1, L):
            forward_cache['Z' + str(l)] = self.networkParameters['W' + str(l)].dot(forward_cache['A' + str(l-1)]) + self.networkParameters['b' + str(l)]
            #forward_cache['Z' + str(l)] = parameters['W' + str(l)].dot(forward_cache['A' + str(l-1)]) + parameters['b' + str(l)]

            if activation == 'tanh':
                forward_cache['A' + str(l)] = self.tanh(forward_cache['Z' + str(l)])
            else:
                forward_cache['A' + str(l)] = self.relu(forward_cache['Z' + str(l)])


        forward_cache['Z' + str(L)] = self.networkParameters['W' + str(L)].dot(forward_cache['A' + str(L-1)]) + self.networkParameters['b' + str(L)]

        if forward_cache['Z' + str(L)].shape[0] == 1:
            forward_cache['A' + str(L)] = self.sigmoid(forward_cache['Z' + str(L)])
        else :
            forward_cache['A' + str(L)] = self.softmax(forward_cache['Z' + str(L)])
        return forward_cache['A' + str(L)], forward_cache

    def compute_cost(self,AL, Y):
        m = Y.shape[1]
        if Y.shape[0] == 1:
            cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
        else:
            cost = -(1./m) * np.sum(Y * np.log(AL))

        cost = np.squeeze(cost)      # To make sure  cost's shape is what we expect.

        return cost

    def backward_propagation(self, AL, Y, forward_cache, activation):
        grads = {}
        L = len(self.networkParameters)//2
        m = AL.shape[1]
        grads["dZ" + str(L)] = AL - Y
        grads["dW" + str(L)] = 1./m * np.dot(grads["dZ" + str(L)],forward_cache['A' + str(L-1)].T)
        grads["db" + str(L)] = 1./m * np.sum(grads["dZ" + str(L)], axis = 1, keepdims = True)
        for l in reversed(range(1, L)):
            if activation == 'tanh':
                grads["dZ" + str(l)] = np.dot(self.networkParameters['W' + str(l+1)].T,grads["dZ" + str(l+1)])*self.derivative_tanh(forward_cache['A' + str(l)])
            else:
                grads["dZ" + str(l)] = np.dot(self.networkParameters['W' + str(l+1)].T,grads["dZ" + str(l+1)])*self.derivative_relu(forward_cache['A' + str(l)])

            grads["dW" + str(l)] = 1./m * np.dot(grads["dZ" + str(l)],forward_cache['A' + str(l-1)].T)
            grads["db" + str(l)] = 1./m * np.sum(grads["dZ" + str(l)], axis = 1, keepdims = True)

        return grads

    def update_parameters(self, grads, learning_rate):

        L = len(self.networkParameters) // 2

        for l in range(L):
            self.networkParameters["W" + str(l+1)] = self.networkParameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
            self.networkParameters["b" + str(l+1)] = self.networkParameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]

    def predict(self, X, y, activation):

        m = X.shape[1]
        y_pred, caches = self.forward_propagation(X, activation)
        if y.shape[0] == 1:
            y_pred = np.array(y_pred > 0.5, dtype = 'float')
        else:
          y = np.argmax(y, 0)
          y_pred = np.argmax(y_pred, 0)
        return np.round(np.sum((y_pred == y)/m), 2)

    def train(self,X,Y,X_test,y_test, learning_rate, num_iteration, activation = 'relu'):
        np.random.seed(1)
        bestTestAcc = 0

        #parameters = initialize_parameters(layers_dims)

        for i in range(0, num_iteration):
            AL, forward_cache = self.forward_propagation(X, activation)
            cost = self.compute_cost(AL, Y)
            grads = self.backward_propagation(AL, Y, forward_cache, activation)
            #print(f"First {grads}")
            self.update_parameters(grads, learning_rate)
            if i % (num_iteration/10) == 0:
                testAcc = self.predict(X_test, y_test, activation)

                #get best Accuracy on test data
                if bestTestAcc < testAcc :
                  bestTestAcc = testAcc
                print("\niter:{} \t cost: {} \t train_acc:{} \t test_acc:{}".format(i, np.round(cost, 2), self.predict(X, Y, activation), testAcc))
            if i % 10 == 0:
                print("==", end = '')

        return bestTestAcc,forward_cache



Import Datasets

In [ ]:
dataset_path = "/content/breast-cancer-wisconsin.data"
dataset_path1 = "/content/car_new.csv"
dataset_path2 = "/content/ecoli.data"
dataset_path3 = "/content/letter-recognition.data"
dataset_path4 = "/content/mushroom.data"

df1 = pd.read_csv(dataset_path) #Breast Cancer Dataset
df2 = pd.read_csv(dataset_path1) #Car Dataset
df3 = pd.read_csv(dataset_path2, sep = '\s+') #Ecoli Dataset
df4 = pd.read_csv(dataset_path3) #Letter Recognition Dataset
df5 = pd.read_csv(dataset_path4) #Mushroom Dataset
df1.head()


#Breast Cancer data Set
df1.columns = ['ID', 'Radius', 'Texture', 'Perimeter', 'Smoothness', 'Compactness', 'Concavity', 'Concave Points', 'Symmetry', 'Fractal Dimension', 'Class']
df1.drop(['ID'], axis = 1, inplace = True)
#df1.head()

#Car
df2.columns = ['Buying', 'Maint', 'Doors', 'Persons', 'Lug_Boots', 'Safety', 'Class']
#df2.head()
#print(df2['Class'])

#E Coli
df3.columns = ['Sequence', 'Mcg', 'Gvh', 'Lip', 'Chg', 'Aac', 'Alm1', 'Alm2', 'Class']
df3.drop(["Sequence"],axis = 1 ,inplace=True)
#df3.head()

#Letter Recognition
df4.columns =['Class','x-box', 'y-box', 'width', 'high',  'onpix', 'x-bar', 'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 'x-ege', 'xegvy' , 'y-ege', 'yegvx']
#df4.head()
#print(df4['Class'])

#Mushroom
df5.columns =['Class','cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']
#df5.head()
#print(df5['Class'])

1) Breast Cancer data Set

In [ ]:
#Covert Concavity from object value to float
df1['Concavity'] = pd.to_numeric(df1['Concavity'], errors='coerce')

# Convert NaN values to zero (optional, depends on your preference)
df1['Concavity'].fillna(0, inplace=True)

# Convert the column to integer type
df1['Concavity'] = df1['Concavity'].astype(int)

# Select Features and Target value
X = df1.iloc[:, :-1] #features
y = df1.iloc[:, -1] #target

y = y.values.reshape(X.shape[0],1)

 # Convert numerical labels to categorical values
y = np.array(y > 2, dtype = 'float')

2) Car Dataset

In [ ]:
#Ecoli data Set
#df1 = pd.read_csv("ecoli_new.csv") #Car Dataset


# Select Features and Target value
X = df2.iloc[:, :-1] #features
y = df2["Class"] #target

# Step 1: Label encode the categorical labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
df_label_encoded = X.copy()
for col in X.columns:
    #print(f"{col}:  {df1[col].dtype}")
    if df2[col].dtype == 'object':  # Only encode categorical columns
        df_label_encoded[col] = label_encoder.fit_transform(df2[col])
X =  df_label_encoded

# Step 2: One-hot encode the integer-encoded labels
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)
print(y.shape)

(1727, 4)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


3) Ecoli data Set

In [ ]:
#Ecoli data Set
#df1 = pd.read_csv("ecoli_new.csv") #Car Dataset


# Select Features and Target value
X = df3.iloc[:, :-1] #features
y = df3.iloc[:, -1] #target
#print(X.shape)
#y=np.squeeze(y)

# Step 1: Label encode the categorical labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
df_label_encoded = X.copy()
for col in X.columns:
    #print(f"{col}:  {df1[col].dtype}")
    if df3[col].dtype == 'object':  # Only encode categorical columns
        df_label_encoded[col] = label_encoder.fit_transform(df1[col])
X =  df_label_encoded

# Step 2: One-hot encode the integer-encoded labels
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)
print(y.shape)

(335, 8)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


4) Letter Recognition Data Set

In [ ]:
#df4.head()

# Select Features and Target value
#print(f"first {df1.head()}")
X = df4.drop("Class", axis=1) #features
y = df4['Class'] #target
#print(y.head())
#print(X.head())

# Get unique values in the 'Column_Name' column
unique_values = y.unique()

# Step 1: Label encode the categorical labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)

# Step 2: One-hot encode the integer-encoded labels
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)
print(y.shape)

(19999, 26)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


5) Mushroom Data

In [ ]:
# Select Features and Target value
X = df5.iloc[:, :-1] #features
y = df5["Class"] #target

#Step 1: Label encode the categorical labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
y = integer_encoded
df_label_encoded = X.copy()
for col in X.columns:
    #print(f"{col}:  {df1[col].dtype}")
    if df5[col].dtype == 'object':  # Only encode categorical columns
        df_label_encoded[col] = label_encoder.fit_transform(df5[col])
X =  df_label_encoded

#Reshape y value
y = y.reshape(X.shape[0],1)
#print(y)
# Convert numerical labels to categorical values
#y = np.array(y > 2, dtype = 'float')

print(y.shape)

(8123, 1)


NN-Declaration

In [ ]:
#Declare Model parameters
layers_dims = [X.shape[1], 40, 40, y.shape[1]] #  4-layer model
lr = 0.01
iters = 100
myNN = NueralNetwork(layers_dims)
#Intialize K-fold
kf = KFold(n_splits=5, shuffle=True, random_state=0)
#Declare Accuracy List
testAccList=[]
for i in range(10):
  for train_index, test_index in kf.split(X):
      # Split data into training and test sets for the current fold
      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      y_train, y_test = y[train_index], y[test_index]

      # # Randomly select 32 samples from the training set
      # X_train_batch = X_train.sample(n=32, random_state=42)
      # y_train_batch = y_train[X_train_batch.index]


      # # Randomly select 32 samples from the training set
      # X_test_batch = X_test.sample(n=32, random_state=42)
      # y_test_batch = y_test[X_test_batch.index]


      X_train_conv = X_train.values
      X_test_conv = X_test.values
      X_train_conv =X_train_conv.T
      X_test_conv = X_test_conv.T
      y_train = y_train.T
      y_test = y_test.T
      #print(f"XTRAIN: {X_train_conv.shape}")
      #print(f"YTRAIN: {y_train.shape}")
      # Train the model
      bestTestAcc,fp = myNN.train(X_train_conv,y_train,X_test_conv,y_test,0.001,1000)
      testAccList.append(bestTestAcc)
      #break;

print(testAccList)
print(f"Average Accuracy is: {np.mean(testAccList)}")
print(f"Standard Deviation for Accuracy is: {np.std(testAccList)}")


iter:0 	 cost: 2.14 	 train_acc:0.67 	 test_acc:0.61
iter:100 	 cost: 0.59 	 train_acc:0.66 	 test_acc:0.66
iter:200 	 cost: 0.53 	 train_acc:0.77 	 test_acc:0.75
iter:300 	 cost: 0.5 	 train_acc:0.8 	 test_acc:0.76
iter:400 	 cost: 0.47 	 train_acc:0.85 	 test_acc:0.8
iter:500 	 cost: 0.46 	 train_acc:0.86 	 test_acc:0.81
iter:600 	 cost: 0.44 	 train_acc:0.87 	 test_acc:0.81
iter:700 	 cost: 0.42 	 train_acc:0.87 	 test_acc:0.82
iter:800 	 cost: 0.41 	 train_acc:0.88 	 test_acc:0.83
iter:900 	 cost: 0.4 	 train_acc:0.89 	 test_acc:0.83
iter:0 	 cost: 0.4 	 train_acc:0.87 	 test_acc:0.89
iter:100 	 cost: 0.39 	 train_acc:0.88 	 test_acc:0.89
iter:200 	 cost: 0.38 	 train_acc:0.88 	 test_acc:0.89
iter:300 	 cost: 0.37 	 train_acc:0.89 	 test_acc:0.89
iter:400 	 cost: 0.36 	 train_acc:0.89 	 test_acc:0.89
iter:500 	 cost: 0.35 	 train_acc:0.89 	 test_acc:0.89
iter:600 	 cost: 0.34 	 train_acc:0.9 	 test_acc:0.91
iter:700 	 cost: 0.34 	 train_acc:0.91 	 test_acc:0.91
iter:800 	 cost: 0.

In [ ]:
len(testAccList)

50